In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer

# ✅ Function to Compare Model Activations with and without Contextual Embeddings
def compare_model_activations(loRa_model, loRa_context_model, tokenizer, sentence):
    """
    Runs a sample sentence through both models:
    1. LoRA-only model (baseline from first fine-tuning)
    2. LoRA + Contextual Embeddings model (second fine-tuning)
    Then, it visualizes the difference in hidden states.
    """

    # ✅ Tokenize Input Sentence
    tokenized_input = tokenizer(sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    input_ids = tokenized_input["input_ids"].to(loRa_model.device)
    attention_mask = tokenized_input["attention_mask"].to(loRa_model.device)

    # ✅ Run Model 1 (LoRA Only - First Fine-Tuning)
    with torch.no_grad():
        output_loRa = loRa_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state

    # ✅ Generate Contextual Embeddings for Model 2
    contextual_embeddings = torch.randn_like(output_loRa) * 0.1  # Simulating external embeddings
    contextual_embeddings = contextual_embeddings.to(loRa_model.device)

    # ✅ Run Model 2 (LoRA + Contextual Embeddings - Second Fine-Tuning)
    with torch.no_grad():
        output_loRa_context = loRa_context_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            contextual_embeds=contextual_embeddings  # ✅ Contextual Embeddings Added
        ).last_hidden_state

    # ✅ Compute Activation Differences
    activation_diff = (output_loRa_context - output_loRa).abs().mean(dim=-1).cpu().numpy()

    # ✅ Visualization
    plt.figure(figsize=(10, 5))
    sns.heatmap(activation_diff, cmap="coolwarm", annot=False)
    plt.xlabel("Token Position")
    plt.ylabel("Batch Sample")
    plt.title("Activation Differences: LoRA Only vs. LoRA + Contextual Embeddings")
    plt.show()

# ✅ Example Usage
# model_loRa = load_fine_tuned_model("XLM-R", use_contextual_embeddings=False)  # First fine-tuned model (LoRA only)
# model_loRa_context = load_fine_tuned_model("XLM-R", use_contextual_embeddings=True)  # Second fine-tuned model (LoRA + Context)
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Test Sentence
# test_sentence = "This is an example sentence to analyze contextual embeddings."

# Compare Model Activations
# compare_model_activations(model_loRa, model_loRa_context, tokenizer, test_sentence)
